# Post-processing

In [1]:
from os import listdir

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
import numpy as np
from pathlib import Path

In [2]:
baseline_df = pd.DataFrame()
for file_path in Path("../results").rglob('baseline_evaluation_results.csv'):
    baseline_df = pd.concat([baseline_df, pd.read_csv(file_path).drop_duplicates()])

ft_df = pd.DataFrame()
for file_path in Path("../results").rglob('evaluation_results.csv'):
    ft_df = pd.concat([ft_df, pd.read_csv(file_path).drop_duplicates()])

In [3]:
baseline_df.head()

,accuracy,auc,eer,precision,recall,f1_score,support,model,model_path,language balance,label balance,fine-tuned languages,evaluated languages,rawboost_algo,fold
0,0.654639,0.641834,0.403065,0.589937,0.937375,0.724138,NaN,rawgat_st,all_models/rawgat_st/weights.pth,{'pl': 3904},"{'spoof': 2000, 'bonafide': 1904}",[],['pl'],0,0
1,0.663015,0.644551,0.398083,0.603965,0.930474,0.732481,NaN,rawgat_st,all_models/rawgat_st/weights.pth,{'pl': 3904},"{'spoof': 2000, 'bonafide': 1904}",[],['pl'],0,1
2,0.649807,0.649158,0.396301,0.589293,0.930647,0.721639,NaN,rawgat_st,all_models/rawgat_st/weights.pth,{'pl': 3904},"{'spoof': 2000, 'bonafide': 1904}",[],['pl'],0,2
3,0.651740,0.639811,0.406034,0.590226,0.933906,0.723317,NaN,rawgat_st,all_models/rawgat_st/weights.pth,{'pl': 3904},"{'spoof': 2000, 'bonafide': 1904}",[],['pl'],0,3
4,0.651740,0.640984,0.401639,0.591845,0.927536,0.722607,NaN,rawgat_st,all_models/rawgat_st/weights.pth,{'pl': 3904},"{'spoof': 2000, 'bonafide': 1904}",[],['pl'],0,4


In [4]:
ft_df.head()

,accuracy,auc,eer,precision,recall,f1_score,support,model,model_path,language balance,label balance,fine-tuned languages,evaluated languages,rawboost_algo,fold
0,0.484858,0.440091,0.534977,0.484081,0.992672,0.650797,NaN,rawgat_st,all_models_1/fine_tuned/rawgat_st_ft_de/weight...,{'pl': 3904},"{'spoof': 2000, 'bonafide': 1904}",['de'],['pl'],0,0
1,0.498389,0.442390,0.535463,0.497074,0.993502,0.662622,NaN,rawgat_st,all_models_1/fine_tuned/rawgat_st_ft_de/weight...,{'pl': 3904},"{'spoof': 2000, 'bonafide': 1904}",['de'],['pl'],0,1
2,0.490335,0.431730,0.536328,0.488954,0.994055,0.655488,NaN,rawgat_st,all_models_1/fine_tuned/rawgat_st_ft_de/weight...,{'pl': 3904},"{'spoof': 2000, 'bonafide': 1904}",['de'],['pl'],0,2
3,0.489691,0.429873,0.546197,0.488478,0.994712,0.655202,NaN,rawgat_st,all_models_1/fine_tuned/rawgat_st_ft_de/weight...,{'pl': 3904},"{'spoof': 2000, 'bonafide': 1904}",['de'],['pl'],0,3
4,0.490657,0.430591,0.544767,0.489769,0.993412,0.656080,NaN,rawgat_st,all_models_1/fine_tuned/rawgat_st_ft_de/weight...,{'pl': 3904},"{'spoof': 2000, 'bonafide': 1904}",['de'],['pl'],0,4


In [5]:
ft_df['fine-tuned languages'] = ft_df['fine-tuned languages'].apply(literal_eval).apply(lambda x: " ".join(x))
baseline_df['evaluated languages'] = baseline_df['evaluated languages'].apply(literal_eval).apply(lambda x: " ".join(x))
ft_df['evaluated languages'] = ft_df['evaluated languages'].apply(literal_eval).apply(lambda x: " ".join(x))
baseline_df['fine-tuned languages'] = '-'

In [6]:
ft_df['model_id'] = ft_df['model'] + " ft with " + ft_df['fine-tuned languages']
baseline_df['model_id'] = "ASV trained " + baseline_df['model']

In [7]:
ft_df['model_id'].unique()

array(['rawgat_st ft with de', 'rawgat_st ft with en',
       'rawgat_st ft with es', 'rawgat_st ft with fr',
       'rawgat_st ft with it', 'rawgat_st ft with pl',
       'rawgat_st ft with ru', 'rawgat_st ft with uk',
       'mesonet ft with de', 'mesonet ft with en', 'mesonet ft with es',
       'mesonet ft with fr', 'mesonet ft with it', 'mesonet ft with pl',
       'mesonet ft with ru', 'mesonet ft with uk',
       'w2v_aasist ft with de', 'w2v_aasist ft with en',
       'w2v_aasist ft with es', 'w2v_aasist ft with fr',
       'w2v_aasist ft with it', 'w2v_aasist ft with pl',
       'w2v_aasist ft with ru', 'w2v_aasist ft with uk',
       'whisper_aasist ft with de', 'whisper_aasist ft with en',
       'whisper_aasist ft with es', 'whisper_aasist ft with fr',
       'whisper_aasist ft with it', 'whisper_aasist ft with pl',
       'whisper_aasist ft with ru', 'whisper_aasist ft with uk'],
      dtype=object)

In [8]:
baseline_df = baseline_df.groupby(['model', 'model_id', 'fine-tuned languages', 'evaluated languages'])[baseline_df.select_dtypes(include=[np.number]).columns].mean()
ft_df = ft_df.groupby(['model', 'model_id', 'fine-tuned languages', 'evaluated languages'])[ft_df.select_dtypes(include=[np.number]).columns].mean()
ft_df.head()

accuracy  \
model   model_id           fine-tuned languages evaluated languages             
mesonet mesonet ft with de de                   de                   0.938520   
                                                en                   0.914103   
                                                es                   0.752679   
                                                fr                   0.978316   
                                                it                   0.917784   

                                                                          auc  \
model   model_id           fine-tuned languages evaluated languages             
mesonet mesonet ft with de de                   de                   0.989377   
                                                en                   0.998168   
                                                es                   0.867421   
                                                fr                   0.999384   
                                                it                   0.992132   

                                                                          eer  \
model   model_id           fine-tuned languages evaluated languages             
mesonet mesonet ft with de de                   de                   0.051095   
                                                en                   0.017676   
                                                es                   0.246723   
                                                fr                   0.007536   
                                                it                   0.044263   

                                                                     precision  \
model   model_id           fine-tuned languages evaluated languages              
mesonet mesonet ft with de de                   de                    0.911253   
                                                en                    0.817671   
                                                es                    0.878651   
                                                fr                    0.961564   
                                                it                    0.864201   

                                                                       recall  \
model   model_id           fine-tuned languages evaluated languages             
mesonet mesonet ft with de de                   de                   0.970056   
                                                en                   0.996642   
                                                es                   0.581329   
                                                fr                   0.995864   
                                                it                   0.987380   

                                                                     f1_score  \
model   model_id           fine-tuned languages evaluated languages             
mesonet mesonet ft with de de                   de                   0.939734   
                                                en                   0.898302   
                                                es                   0.699691   
                                                fr                   0.978412   
                                                it                   0.921693   

                                                                     support  \
model   model_id           fine-tuned languages evaluated languages            
mesonet mesonet ft with de de                   de                       NaN   
                                                en                       NaN   
                                                es                       NaN   
                                                fr                       NaN   
                                                it                       NaN   

                                                                     rawboost_algo  \
model  

In [9]:
baseline_df.head()

accuracy  \
model     model_id              fine-tuned languages evaluated languages             
rawgat_st ASV trained rawgat_st -                    de                   0.517921   
                                                     en                   0.478686   
                                                     es                   0.454592   
                                                     fr                   0.562436   
                                                     it                   0.462951   

                                                                               auc  \
model     model_id              fine-tuned languages evaluated languages             
rawgat_st ASV trained rawgat_st -                    de                   0.527280   
                                                     en                   0.582452   
                                                     es                   0.454836   
                                                     fr                   0.585438   
                                                     it                   0.454264   

                                                                               eer  \
model     model_id              fine-tuned languages evaluated languages             
rawgat_st ASV trained rawgat_st -                    de                   0.467358   
                                                     en                   0.453133   
                                                     es                   0.545108   
                                                     fr                   0.441080   
                                                     it                   0.534851   

                                                                          precision  \
model     model_id              fine-tuned languages evaluated languages              
rawgat_st ASV trained rawgat_st -                    de                    0.507791   
                                                     en                    0.416784   
                                                     es                    0.449395   
                                                     fr                    0.535390   
                                                     it                    0.455214   

                                                                            recall  \
model     model_id              fine-tuned languages evaluated languages             
rawgat_st ASV trained rawgat_st -                    de                   0.795174   
                                                     en                   0.924238   
                                                     es                   0.445365   
                                                     fr                   0.855610   
                                                     it                   0.487314   

                                                                          f1_score  \
model     model_id              fine-tuned languages evaluated languages             
rawgat_st ASV trained rawgat_st -                    de                   0.619788   
                                                     en                   0.574464   
                                                     es                   0.447339   
                                                     fr                   0.658637   
                                                     it                   0.470695   

                                                                          support  \
model     model_id              fine-tuned languages evaluated languages            
rawgat_st ASV trained rawgat_st -                    de                       NaN   
                                                     en                       NaN   
                                                     es                       NaN   
                        

In [10]:
df = pd.concat([baseline_df, ft_df])
df

accuracy  \
model          model_id                  fine-tuned languages evaluated languages             
rawgat_st      ASV trained rawgat_st     -                    de                   0.517921   
                                                              en                   0.478686   
                                                              es                   0.454592   
                                                              fr                   0.562436   
                                                              it                   0.462951   
...                                                                                     ...   
whisper_aasist whisper_aasist ft with uk uk                   fr                   0.563074   
                                                              it                   0.537178   
                                                              pl                   0.554639   
                                                              ru                   0.500586   
                                                              uk                   0.585374   

                                                                                        auc  \
model          model_id                  fine-tuned languages evaluated languages             
rawgat_st      ASV trained rawgat_st     -                    de                   0.527280   
                                                              en                   0.582452   
                                                              es                   0.454836   
                                                              fr                   0.585438   
                                                              it                   0.454264   
...                                                                                     ...   
whisper_aasist whisper_aasist ft with uk uk                   fr                   0.594752   
                                                              it                   0.574705   
                                                              pl                   0.583962   
                                                              ru                   0.524868   
                                                              uk                   0.599382   

                                                                                        eer  \
model          model_id                  fine-tuned languages evaluated languages             
rawgat_st      ASV trained rawgat_st     -                    de                   0.467358   
                                                              en                   0.453133   
                                                              es                   0.545108   
                                                              fr                   0.441080   
                                                              it                   0.534851   
...                                                                                     ...   
whisper_aasist whisper_aasist ft with uk uk                   fr                   0.416688   
                                                              it                   0.440106   
                                                              pl                   0.429980   
                                                              ru                   0.480080   
                                                              uk                   0.403441   

                                                                                   precision  \
model          model_id                  fine-tuned languages evaluated languages              
rawgat_st      ASV trained rawgat_st     -                    de                    0.507791   
                                                              en                    0.4

In [11]:
df = df.apply(lambda x: x).reset_index()

In [12]:
df.pivot(index=['model_id'], columns='evaluated languages', values=['eer'])

eer                                          \
evaluated languages               de        en        es        fr        it   
model_id                                                                       
ASV trained rawgat_st       0.467358  0.453133  0.545108  0.441080  0.534851   
ASV trained whisper_aasist  0.463953  0.459645  0.498789  0.450692  0.449843   
mesonet ft with de          0.051095  0.017676  0.246723  0.007536  0.044263   
mesonet ft with en          0.072406  0.011236  0.280476  0.005169  0.033396   
mesonet ft with es          0.072445  0.029564  0.148495  0.009459  0.041151   
mesonet ft with fr          0.079751  0.018134  0.308147  0.004111  0.028167   
mesonet ft with it          0.069906  0.007838  0.084618  0.003088  0.014382   
mesonet ft with pl          0.070369  0.011447  0.206695  0.004394  0.018744   
mesonet ft with ru          0.270378  0.355171  0.474127  0.269163  0.341230   
mesonet ft with uk          0.091180  0.063059  0.285484  0.009696  0.054278   
rawgat_st ft with de        0.424642  0.273604  0.485583  0.430788  0.414769   
rawgat_st ft with en        0.592940  0.646608  0.617711  0.616949  0.624396   
rawgat_st ft with es        0.462346  0.383382  0.397578  0.496681  0.466093   
rawgat_st ft with fr        0.496554  0.210635  0.587297  0.424556  0.438898   
rawgat_st ft with it        0.549949  0.640835  0.563544  0.592924  0.545466   
rawgat_st ft with pl        0.457503  0.311405  0.367587  0.399985  0.413542   
rawgat_st ft with ru        0.481301  0.366944  0.569416  0.462759  0.495984   
rawgat_st ft with uk        0.564107  0.610046  0.471125  0.475983  0.561044   
w2v_aasist ft with de       0.325566  0.360771  0.319862  0.277017  0.269143   
w2v_aasist ft with en       0.599776  0.647693  0.520677  0.590622  0.587277   
w2v_aasist ft with es       0.206374  0.202653  0.174984  0.162808  0.183023   
w2v_aasist ft with fr       0.172615  0.198430  0.245865  0.172258  0.134550   
w2v_aasist ft with it       0.603286  0.573288  0.439809  0.602042  0.523748   
w2v_aasist ft with pl       0.191753  0.289615  0.322065  0.232002  0.144541   
w2v_aasist ft with ru       0.613088  0.580687  0.609099  0.570113  0.609530   
w2v_aasist ft with uk       0.468229  0.489913  0.315084  0.577661  0.504116   
whisper_aasist ft with de   0.434509  0.431218  0.497004  0.419874  0.431238   
whisper_aasist ft with en   0.449370  0.439594  0.502103  0.434465  0.441180   
whisper_aasist ft with es   0.436272  0.426522  0.487045  0.415472  0.434944   
whisper_aasist ft with fr   0.448320  0.437826  0.496367  0.417736  0.427002   
whisper_aasist ft with it   0.438413  0.420558  0.491906  0.420252  0.430634   
whisper_aasist ft with pl   0.435390  0.420870  0.488464  0.409961  0.422502   
whisper_aasist ft with ru   0.436524  0.442640  0.480178  0.416981  0.433488   
whisper_aasist ft with uk   0.440310  0.432609  0.494073  0.416688  0.440106   

                                                          
evaluated languages               pl        ru        uk  
model_id                                                  
ASV trained rawgat_st       0.401024  0.463524  0.356818  
ASV trained whisper_aasist  0.455271  0.479693  0.445930  
mesonet ft with de          0.010828  0.049827  0.000000  
mesonet ft with en          0.010155  0.072569  0.000000  
mesonet ft with es          0.019238  0.021481  0.000000  
mesonet ft with fr          0.005806  0.038909  0.000000  
mesonet ft with it          0.010327  0.024458  0.000000  
mesonet ft with pl          0.004149  0.059371  0.000000  
mesonet ft with ru          0.224289  0.126655  0.001048  
mesonet ft with uk          0.007939  0.055782  0.000000  
rawgat_st ft with de        0.539546  0.459848  0.465208  
rawgat_st ft with en        0.707713  0.534028  0.478124  
rawgat_st ft with es        0.508559  0.457519  0.434750  
rawgat_st ft with fr        0.655482  0.545599  0.609536  
rawgat_st ft with it        0.650005  0.454648  0.407948  
rawgat_st 

In [ ]:
def create_heatmap(dataframe, model_name, metric='eer', metric_label="EER"):
    heatmap_data = dataframe.loc[dataframe['model'] == model_name].pivot(index='fine-tuned languages', columns='evaluated languages', values=metric)
    plt.figure(figsize=(10, 8))
    ax = sns.heatmap(
        heatmap_data,
        annot=True,
        cmap="Blues",
        cbar_kws={'label': metric_label},
        annot_kws={"size": 16},
        fmt='.2f'
    )
    plt.xlabel('Evaluated with', fontsize=16)
    plt.ylabel('Fine-Tuned With', fontsize=16)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    if '-' in heatmap_data.index:
        idx = heatmap_data.index.get_loc('-')
        n_rows = len(heatmap_data.index)
        n_cols = len(heatmap_data.columns)
        
        # Draw horizontal lines (top and bottom of the frame)
        ax.add_patch(plt.Rectangle((0, idx), n_cols, 1, fill=False, edgecolor='black', lw=4))
        
        # Draw vertical lines (left and right of the frame)
        ax.vlines(x=[0, n_cols], ymin=n_rows-idx-1, ymax=n_rows-idx, colors='black', linewidth=4)
        
    model_name = model_name.lower()
    if 'rawgat_st' in model_name:
        model_name = model_name.replace('rawgat_st', 'RawGAT ST')
    if 'w2v_aasist' in model_name:
        model_name = model_name.replace('w2v_aasist', 'W2V + AASIST')
    elif 'mesonet' in model_name:
        model_name = model_name.replace('mesonet', 'LFCC + MesoNet')
    if 'whisper_aasist' in model_name:
        model_name= model_name.replace('whisper_aasist', 'Whisper + AASIST')
    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=16)
    cbar.set_label(metric_label, fontsize=16)
    plt.title(f'{metric_label} for {model_name}', fontsize=18)
    plt.savefig(f"../results/plots/{model_name.lower().replace('+', '-')}.png")
    plt.show()

In [ ]:
for name in df.apply(lambda x: x)['model'].unique():
    create_heatmap(df, name)


In [ ]:
for name in df.apply(lambda x: x)['model'].unique():
    create_heatmap(df, name, "accuracy", "Accuracy")